# recupe des dossier depuis api lien 

In [1]:
import requests
import os
from datetime import datetime
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

def download_csv_file():
    # URL de l'API
    url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/bornes-irve/exports/csv"
    
    # Paramètres de la requête
    params = {
        'lang': 'fr',
        'refine': 'region:"Hauts-de-France"',
        'facet': 'facet(name="region", disjunctive=true)',
        'timezone': 'Europe/Berlin',
        'use_labels': 'true',
        'delimiter': ';'
    }

    try:
        # Créer le dossier data s'il n'existe pas
        if not os.path.exists('data'):
            os.makedirs('data')

        # Faire la requête HTTP
        response = requests.get(url, params=params)
        response.raise_for_status()  # Lever une exception si la requête échoue

        # Générer un nom de fichier avec la date
        current_date = datetime.now().strftime("%Y%m%d")
        filename = f'data/bornes_irve_hdf_{current_date}.csv'

        # Sauvegarder le fichier
        with open(filename, 'wb') as f:
            f.write(response.content)

        print(f"Fichier téléchargé avec succès: {filename}")
        return filename

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement: {e}")
        return None

if __name__ == "__main__":
    downloaded_file = download_csv_file()


Fichier téléchargé avec succès: data/bornes_irve_hdf_20250129.csv


# envoie du fichier dans le Data Lake

In [3]:
import os
from azure.storage.filedatalake import DataLakeServiceClient
from datetime import datetime
from dotenv import load_dotenv

def upload_to_datalake(local_file_path):
    # Charger les variables d'environnement
    load_dotenv()

    # Récupérer les informations de connexion
    storage_account_name = os.getenv('STORAGE_ACCOUNT_NAME')
    storage_account_key = os.getenv('STORAGE_ACCOUNT_KEY')
    container_name = os.getenv('CONTAINER_NAME')

    try:
        # Créer l'URL du service
        service_client = DataLakeServiceClient(
            account_url=f"https://{storage_account_name}.dfs.core.windows.net",
            credential=storage_account_key
        )

        # Obtenir une référence au système de fichiers (container)
        file_system_client = service_client.get_file_system_client(container_name)

        # Définir le chemin dans le Data Lake
        # Par exemple: bornes-irve/2024/01/fichier.csv
        current_date = datetime.now()
        datalake_path = f"bornes-irve/{current_date.year}/{current_date.month:02d}/{os.path.basename(local_file_path)}"

        # Obtenir une référence au fichier
        file_client = file_system_client.get_file_client(datalake_path)

        # Upload le fichier
        with open(local_file_path, 'rb') as file:
            file_contents = file.read()
            file_client.upload_data(file_contents, overwrite=True)

        print(f"Fichier uploadé avec succès vers: {datalake_path}")
        return True

    except Exception as e:
        print(f"Erreur lors de l'upload vers Data Lake: {e}")
        return False

if __name__ == "__main__":
    # Utiliser le dernier fichier téléchargé
    current_date = datetime.now().strftime("%Y%m%d")
    local_file = f'data/bornes_irve_hdf_{current_date}.csv'
    
    if os.path.exists(local_file):
        upload_to_datalake(local_file)
    else:
        print(f"Fichier local non trouvé: {local_file}")


Fichier uploadé avec succès vers: bornes-irve/2025/01/bornes_irve_hdf_20250129.csv
